In [8]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../')) # or the path to your source code
sys.path.insert(0, module_path)

In [9]:
import pandas as pd 
import tiledbsoma as soma
import tiledb

import pickle

from bascvi.datamodule.soma.soma_helpers import open_soma_experiment

/home/ubuntu/miniconda3/envs/bascivi/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# read soma 
with open_soma_experiment("s3://pai-scrnaseq/sctx_gui/corpora/scTx-v1.3.mm/") as soma_experiment:
    obs_df = soma_experiment.obs.read().concat().to_pandas()
    print(obs_df.shape)

In [33]:
obs_df.head()

,soma_joinid,dataset_idx,sample_idx,barcode,standard_true_celltype,authors_celltype,age_stage,batch_display,batch_name,cell_type_enrichment,...,sample_name,scrnaseq_protocol,study_name,tissue_collected,tissue_dissociation,tissue_primary,tissue_site,treatment,nnz,umi
0,0,0,0,21784_6-1118_tumor_21784-6-121,,,Adult,B16.8d (Davidson2020 - Tumor),cancer.skin,separate-immune-and-stroma,...,21784_6-1118_tumor,Smart-seq2,external_m-davidson_cellrep_2020_32433953,Skin,enzymatic,Skin,Primary,None reported,1136,14377
1,1,0,0,21784_6-1118_tumor_21784-6-123,,,Adult,B16.8d (Davidson2020 - Tumor),cancer.skin,separate-immune-and-stroma,...,21784_6-1118_tumor,Smart-seq2,external_m-davidson_cellrep_2020_32433953,Skin,enzymatic,Skin,Primary,None reported,2383,254223
2,2,0,0,21784_6-1118_tumor_21784-6-125,,,Adult,B16.8d (Davidson2020 - Tumor),cancer.skin,separate-immune-and-stroma,...,21784_6-1118_tumor,Smart-seq2,external_m-davidson_cellrep_2020_32433953,Skin,enzymatic,Skin,Primary,None reported,3548,740308
3,3,0,0,21784_6-1118_tumor_21784-6-127,,,Adult,B16.8d (Davidson2020 - Tumor),cancer.skin,separate-immune-and-stroma,...,21784_6-1118_tumor,Smart-seq2,external_m-davidson_cellrep_2020_32433953,Skin,enzymatic,Skin,Primary,None reported,4139,802568
4,4,0,0,21784_6-1118_tumor_21784-6-129,,,Adult,B16.8d (Davidson2020 - Tumor),cancer.skin,separate-immune-and-stroma,...,21784_6-1118_tumor,Smart-seq2,external_m-davidson_cellrep_2020_32433953,Skin,enzymatic,Skin,Primary,None reported,2347,456209


In [35]:
obs_df[['batch_display']].drop_duplicates().shape

(241, 1)

In [36]:
# downsample to 10k cells per batch oclumns
obs_df_sub = obs_df.groupby("batch_display").apply(lambda x: x.sample(n=10000) if x.shape[0] > 10000 else x).reset_index(drop=True)
print(obs_df_sub.batch_display.value_counts())
print(obs_df_sub.shape)

EMT6.Baseline-T (Swarbrick2022)                        10000
p53 primary sarcoma Isotype (Wisdom2020 - Imm)         10000
p53 primary sarcoma Isotype 20Gy (Wisdom2020 - Imm)    10000
4T1.IgG-control-LN (Swarbrick2022)                     10000
4T1.IgG-control-T (Swarbrick2022)                      10000
                                                       ...  
B16F10.untr (Anandhan2024 - NoImm)                       524
B16F10.untr (Kumar2018 - Imm)                            422
B16F10.untr (Kumar2018)                                  246
B16.0d (Davidson2020 - LN)                               123
B16.0d (Davidson2020 - Tumor)                            120
Name: batch_display, Length: 241, dtype: int64
(994155, 34)


In [38]:
# save barcodes 
with open("/home/ubuntu/ml/bascvi/data/v1.3mm_train_barcodes.pkl", "wb") as f:
    pickle.dump(obs_df_sub.barcode.to_list(), f)